In [65]:
# Функция для очистки данных(без удаления nan)
import time
def clearing(data):
    import pandas as pd
    import re
    import ast
    top = ['WA', 'NH', 'MN', 'UT', 'VT', 'MD', 'VA', 'MA', 'NE', 'CO', 'WI', 'NJ', 'FL', 'IA', 'ND', 'ID']
    mid = ['GA', 'NC', 'CA', 'SD', 'CT', 'KS', 'DE', 'HI', 'NY', 'RI', 'OR', 'MO', 'MT', 'TN', 'WY', 'ME']
    data = data.drop(['MlsId', 'mls-id', 'private pool', 'PrivatePool', 'propertyType', 'status', 'fireplace', 'target', 'state'],axis = 1)

    def func_pool(x):
        if type(x) != int:
            x = x.replace(x, '1')
        return int(x)
    
    def func_stor(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if (len(x) > 1):
                x = x[0]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 1
        return x
    
    def func_area(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if len(x) > 1:
                x = x[0] + x[1]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 4940
        return int(x)

    def func_lot(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if len(x) > 1:
                x = x[0] + x[1]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 5200
        return int(x)
    
    def func_built(x):
        x = re.findall(r'\d+',x)
        if len(x) > 1:
            x = x[0] + x[1]
        elif len(x) == 1:
            x = x[0]
        if not x:
            return 1976
        return int(x)
        
    def data_func1(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][0].get('factValue')

    def data_func6(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][5].get('factValue')
    
    def data_func8(x):
        m = []
        a = ast.literal_eval(x)
        a = dict(a[0])
        k = a['data'].get('Distance')
        for i in k:
            h = re.findall(r'\d+',i)
            if len(h) == 0:
                continue
            if len(h) == 1:
                m.append(float(h[0]))
                continue
            d = h[0] + '.' + h[1]
            m.append(float(d))
        if len(m) == 0:
            return 1.1
        else:
            return min(m)
 
    data['baths'] = data['baths'].fillna(1).apply(func_stor)
    data['stories'] = data['stories'].fillna(1).apply(func_stor)
    data['beds'] = data['beds'].fillna(1).apply(func_stor)
    data['built'] = data['homeFacts'].apply(data_func1)
    data['lotsize'] = data['homeFacts'].apply(data_func6)
    data['distance'] = data['schools'].apply(data_func8)
    data = data.drop(['homeFacts', 'schools'], axis = 1)
    data['lotsize'] = data['lotsize'].fillna(5200).apply(func_lot)
    data['built'] = data['built'].fillna(1976)
    data['built'] = data['built'].astype(str)
    data['built'] = data['built'].apply(func_built)
    data['built'] = data['built'].astype(int)
    data['sqft'] = data['sqft'].fillna(4940).apply(func_area)
    data['baths'] = data['baths'].astype(int)
    data['beds'] = data['beds'].astype(int)
    data['stories'] = data['stories'].astype(int)
    return data

# Загрузка модели
with open('mymodel3.pkl', 'rb') as pkl_file:
    regressor = pickle.load(pkl_file)

In [67]:
%%time
# Импорт библиотек
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Считывание и обработка данных
filename = 'data.csv'
data = pd.read_csv(filename)
data = data[:10000]
data = clearing(data)

# Загрузка обработанной таблицы для корректной работы LabelEncoder
totrain = pd.read_csv('obr.csv')
totrain = totrain.drop(['heating', 'cooling', 'parking', 'PrivatePool', 'target', 'fireplace', 'state', 'rebuilt'], axis = 1)

# Удаление строк без адресса
data = data.drop(data.loc[data['city'].isnull()].index)
data = data.drop(data.loc[data['street'].isnull()].index)
data = data.drop(data.loc[data['zipcode'].isnull()].index)

# Соединение двух таблиц и обработка категориальных признаков
full = pd.concat([data, totrain], ignore_index=True)
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []
features = full.columns.values.tolist()
for col in features:
    if full[col].dtype in numerics: continue
    categorical_columns.append(col)
for col in categorical_columns:
    if col in full.columns:
        le = LabelEncoder()
        le.fit(list(full[col].astype(str).values))
        full[col] = le.transform(list(full[col].astype(str).values))
    
# Отделение обработанных данных для предсказания
X = full[:len(data)]

# Предсказание и обработка результата
pred = regressor.predict(X)
for x in range(len(pred)):
    pred[x] = round(pred[x]/1000)*1000
target = pd.DataFrame(pred, columns = ['target'])
final = X.join(target)

# Итог
final.head()

Wall time: 15.6 s


,street,baths,city,sqft,zipcode,beds,stories,built,lotsize,distance,target
0,72110,3,1320,2900,942,4,1,2019,5200,2.70,586000.0
1,24754,3,1328,1947,4059,3,2,2019,5828,1.01,310000.0
2,58803,2,806,3000,3240,3,1,1961,8626,1.19,2276000.0
3,114489,8,341,6457,2360,5,3,2006,8220,0.10,1992000.0
4,39303,1,1034,4940,1194,1,1,1976,10019,3.03,1278000.0
